In [1]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np

In [16]:
streets = pd.read_csv('../data/streets.csv')

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3,5,6,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
groups = streets.groupby('postcode')

In [20]:
cleaned = []
for m_id, values in groups:
    city = []
    for nl, fr in values[['streetname_nl', 'streetname_fr']].values:
        try:
            if nl is not np.nan:
                city.append(nl.lower())
        except:
            print(nl, fr)
        try:
            if fr is not np.nan:
                city.append(fr.lower())
        except:
            print(nl, fr)
    cleaned.append(city)

nan Rue de la Cabourse
nan allée du Bois des Rêves
nan rue de la Brulotte
nan rue du Bois des Rêves
nan cour des Terres Noires
nan scavée du Biéreau
nan Rue de la Queue de Geest
nan Allée Des Ormes
nan Hameau de la Gette 
nan Rue de l'Ermite
nan Venelle de Barbançon
nan Avenue des Vallées
nan Rue de l'Industrie
nan Clos du Diable Vert
nan Venelle de Froidevau
nan Chemin de Baudémont
nan Allée Marcel Ladrière
nan Avenue du 21 Juillet
nan Rue des Communes
nan Rue Ernest Solvay
nan Place Jules-Seeliger
nan Place Marcel-Lonay
nan Place Notger
nan Place Reine-Astrid
nan Place Saint-Barthélemy
nan Place Saint-Christophe
nan Place Saint-Denis
nan Place Sainte-Véronique
nan Place Sainte-Walburge
nan Place Saint-Jacques
nan Place Saint-Lambert
nan Quai de l'Ile-aux-Osiers
nan Rue Beeckman
nan Quai Saint-Léonard
nan Rue César-Franck
nan Rue Chapeauville
nan Rue Chapelle-des-Clercs
nan En Neuvice
nan Rue André-Winands
nan Rue Auguste Buisseret
nan Rue Auguste-Donnay
nan Rue Auguste-Doutrepont
nan

nan Sart aux Fraises
nan Square de la Paix-d'Angleur
nan Square des Conduites-d'Eau
nan Rue Saint-Jacques
nan Rue Henri Defêchereux
nan Rue Henri-Dunant
nan Rue Sous-le-Bois
nan Traverse des Architectes
nan Rue Joseph-Marcotty
nan Rue Renkin
nan Rue Romaine
nan Rue Ovide-Decroly
nan Rue Triolet
nan Rue Jules-Verne
nan Rue Pré-Cala
nan Rue Sart-Tilman
nan Rue Claude Strebelle
nan Place Joseph-Willem
nan Rue Chaffette
nan Boulevard de l'Ourthe
nan Aux Piedroux
nan Parc Sauveur
nan Cité des Thiers
nan Cité du Réveil
nan Quai des Ardennes
nan Rue Bois-Guéau
nan Impasse du Gué
nan Rue Bourdon
nan Place du Tilleul
nan Place du Vieux-Puits
nan Rue Chevy
nan Cour Lempereur
nan Rue Alban-Poulet
nan Place Hubert-Pissard
nan Quai Henri-Borguet
nan Rue André-Renard
nan Allée Edgard-D'Hont
nan Cour Poupier
nan Cour Tassin
nan Rue des Mauvaises-Vignes
nan Rue de la Rainette
nan Rue de la Reconnaissance
nan Rue de la Révision
nan Rue de l'Aviation
nan Rue de l'Eglise
nan Rue de l'Epargne
nan Rue de S

nan Rue des Sapins
nan Rue du Centre
nan Rue du Christ
nan Rue du Petit Bioleux
nan Rue du Ruisseau
nan Rue du Thier
nan Rue Fine Pierre
nan Rue Grande Terre
nan Rue Grandzée
nan Rue Grosse Pierre
nan Rue de la Limite
nan Rue Auguste Donnay
nan Rue Baory
nan Rue Baoufontaine
nan Rue Bayfils
nan Rue Bégasse
nan Rue Blandot
nan Rue Bois des Manants
nan Rue Bovière
nan Rue Chamelot
nan Rue Chanteclair
nan Rue Chevalier P. de Sauvage
nan Avenue des Ardennes
nan Avenue des Bois
nan Avenue des Bouleaux
nan Avenue des Ormes
nan Avenue des Trois Couronnes
nan Rue de l'Ancien Vélodrome
nan Rue de l'Athénée
nan Rue de Liège
nan Rue de l'Ile
nan Avenue d' Esneux
nan Avenue du Midi
nan Avenue du Monceau
nan Rue des Hêtres
nan Rue des Mésanges
nan Rue des Messes
nan Rue des Nutons
nan Domaine du Pont de Méry
nan Esplanade de l'Abeille
nan Evieux
nan Rue de Limoges
nan Rue Fond du Moulin
nan Rue Fond Noé
nan Rue Foxhalle
nan Rue Fraipont
nan Rue Fréson
nan Rue Gaston Bernard
nan Rue des Pins
nan Rue

nan Rue Sainte Apolline
nan Rue Saint Etienne
nan Rue Saint Lambert
nan Rue Saint Martin
nan Rue Saint Remy
nan Rue sous les Monts
nan Rue sous les Prés
nan Rue Tige de Moxhe
nan Rue Tige Jacquette
nan Rue Neuve
nan Rue de Malpas
nan Rue de la Station
nan Rue de la Vallée
nan Rue de Lacosse
nan Rue de Landen
nan Rue de l'Arbre
nan Rue de l'Eglise
nan Rue de Liège
nan Rue de Linsmeau
nan Rue de Lussac
nan Rue de l'Yser
nan Rue de Maret
nan Rue de Neerheylissem
nan Rue de Pellaines
nan Rue de Jodoigne
nan Rue Bénédicale
nan Rue d'Avernas
nan Rue de Grand'Hallet
nan Rue de la Bacquelaine
nan Rue de la Bruyère
nan Rue de la Fontaine
nan Rue de la Gironde
nan Rue de la Petite Gèthe
nan A la Courte Terre
nan Aux Pirées
nan Avenue des Français
nan Avenue des Sorbiers
nan Chemin de Lincent
nan Chemin des Prés
nan Clos de la Drève
nan Place Adelin Lheureux
nan Plèce Saint-Christophe
nan Route de Huy
nan Rue du Bailly
nan Ruelle Bovy
nan Ruelle des Prêtres
nan Ruelle Dongleur
nan Ruelle du Maréc

nan Allée de la Santé
nan Chaussée de Tongres
nan Place Carton Denrart
nan Rue des Oeillets
nan Rue des Pinsons
nan Rue Provinciale
nan Rue de la Renaissance
nan Rue du Cimetière
nan Rue Vieille Voie de Tongres
nan Rue Voie de Liège
nan Ruelle du Bois
nan Rue du Pairoux
nan Rue des Aubépines
nan Rue du Vieux Moulin
nan Al Bail
nan Chaussée Brunehault
nan Chaussée Brunehaut
nan Chaussée de Tongres
nan Clos Fleuri
nan Clos Fredman
nan Clos Mosan
nan La Niestrée
nan Passage de la Béguine
nan Route de Glons
nan Rue Bourgogne
nan Rue de Charleroi
nan Rue des Hayettes
nan Rue Lambert Dewonck
nan Rue d'Heure-le-Tixhe
nan Rue de la Bourgogne
nan Rue du Centenaire
nan Rue de la Seigneurie
nan Rue de Paifve
nan Rue Emile Grisard
nan Rue entre deux Prés
nan Voie de l'Espinette
nan Chaussée Brunehaut
nan Rue Guillaume Maréchal
nan Rue Lambert Tilkin
nan Rue Saint-Lambert
nan Rue du Docteur Keppenne
nan Rue du Tige
nan Rue des Bleuets
nan Clos des Huit Bonniers
nan Clos Nicolas Colette
nan Rue Anix

...

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


nan Voie d'Honoré Champs
nan Coingsoux
nan Rue Robert Peutat
nan Rue Trixhes aux Rames
nan Rue Vesdray
nan Rue de la Pierresse
nan Rue de l'Ecole
nan Rue de l'Eglise
nan Rue de l'Invasion
nan Rue des Déportés
nan Rue des Massacres
nan Rue du Monument
nan Rue Freux Prés
nan Rue Gaspard Oger
nan Rue Guillaume Gerards
nan Rue Hauglustaine
nan Rue Henri Theek
nan Rue Jean Wansart
nan Nantistay
nan Pirhettes
nan Rue Chaffour
nan Rue de Béthane
nan Rue de Hestreux
nan Rue de Hèvremont
nan Rue de la Gileppe
nan Runschen
nan Sur les Roches
nan Vreuschemen
nan Chemin des Passeurs
nan Levée de Limbourg
nan Rue des Fusillés
nan Rue des Millepertuis
nan Rue des Pâquerettes
nan Rue des Tilleuls
nan Rue du Développement
nan Chemin du Giesberg
nan Chemin du Ruyff
nan Chemin Kyssel
nan Loussack
nan Mazarinen
nan Médael (Elzet)
nan Rue du Moulin
nan Rue du Pensionnat
nan Meuschemen
nan Rue du Thier
nan Rue du Vicinal
nan Rue Emile Schmuck
nan Rue Hubert Braun
nan Rue Jean Renardy
nan Rue Joseph Cardijn

nan Rue Maurice Pottier
nan Rue Pré Jonas
nan Rue Promenade de Quatre Heures
nan Rue Renesse
nan Rue Rogier
nan Rue Royale
nan Rue Sandberg
nan Rue Servais
nan Avenue Antoine Pottier
nan Avenue Bel Air
nan Avenue Bel Heid
nan Avenue Camille Bellenger
nan Avenue Chevalier De Thier
nan Avenue Clémentine
nan Avenue Marie-Thérèse
nan Avenue Messidor
nan Avenue de la Bovière
nan Avenue de la Corniche
nan Parc Docteur Jean Barzin
nan Parc Reine Elisabeth
nan Petite Ruelle du Tri-Renard
nan Place Achille Salée
nan Place de la Providence
nan Place des Ecoles
nan Place du Monument
nan Place du Perron
nan Place Pierre le Grand
nan Avenue Peltzer de Clermont
nan Avenue Princesse Clémentine
nan Avenue Professeur Henrijean
nan Place Royale
nan Place Verte
nan Préfayhai
nan Promenade Berkeley
nan Rue de la Sauvenière
nan Rue de l'Abattoir
nan Rue de l'Eglise
nan Rue de l'Hôtel de Ville
nan Rue de Sclessin
nan Avenue de la Havette
nan Avenue des Aubépines
nan Avenue Reine Astrid
nan Avenue des Bouton

nan Rue d'Alvaux
nan Fosses-aux-Chênes
nan Rue de Trehet
nan Rue du Grand Saule
nan Rue Forêt
nan Rue du Bois Planté
nan Rue de Gilberoux
nan rue du Presbytère
nan Rue de Saint-Gobain
nan Chemin du Bois du Roi
nan Rue Haute Baive
nan Rue Romaine
nan Tiège du Moulin
nan Roissia
nan Rue du Batis du Moulin
nan Rue Fernand Bourgeois
nan Rue Saint-Raboni
nan Rue de la Marsale
nan Chemin de Tibiéré
nan Rue de la Salle
nan rue de la Navère
nan Rue Marot
nan Rue des Claviats
nan Rue des Russes
nan Rue Auguste, D.-M.-F.
nan Rue Désiré, D.-M.-F.
nan Rue Renée, D.-M.-F.
nan Rue Roger, D.-M.-F.
nan Rue de Caneva
nan Chemin Barons de Rosée
nan Rue du Rivage
nan Rue d'Anthée (Feron)
nan Rue de l'Eglise (WT)
nan Domaine-Montmeuse
nan Rue de la Sapinière
nan Le-Crupet
nan Rue Larifosse
nan Rue de Meuse, Les Pauquis
nan Le Cortil du Meunier
nan Chemin Le Crupet
nan Ferme-de-Tahaut
nan Fosse-Dondaine
nan Ruelle Raclot
nan Chemin du Halage
nan Quatre Chemins
nan Allée des Tourterelles (Relax)
nan Allée d

nan Clos de la Maraude
nan Place du Marché
nan Rue Le Chat
nan Grand Chemin
nan Rue de la Chapelle


In [35]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_model = Word2Vec(min_count=10,
                     window=10,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)



In [36]:
w2v_model.build_vocab(cleaned, progress_per=10000)

In [37]:
w2v_model.train(cleaned, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(297212, 4461480)

In [38]:
w2v_model.init_sims(replace=True)

In [39]:


w2v_model.wv.most_similar(positive=["kerkstraat"])



[('molenstraat', 0.9999303817749023),
 ('schoolstraat', 0.9998067021369934),
 ('veldstraat', 0.9995279312133789),
 ('nieuwstraat', 0.9994289875030518),
 ('stationsstraat', 0.9987086057662964),
 ('kloosterstraat', 0.9986938834190369),
 ('merelstraat', 0.9973377585411072),
 ('zandstraat', 0.9963194727897644),
 ('hoogstraat', 0.9962786436080933),
 ('groenstraat', 0.9951494932174683)]

In [40]:
w2v_model.save('models/streets_model_2.wv')

In [28]:
w2v_model.wv.most_similar(positive=["dorpsstraat"])

[('bergstraat', 0.999914288520813),
 ('kerkhofstraat', 0.9999118447303772),
 ('broekstraat', 0.9999117851257324),
 ('acacialaan', 0.999910295009613),
 ('zandstraat', 0.9999096393585205),
 ('rozenlaan', 0.9999072551727295),
 ('spoorwegstraat', 0.9999048113822937),
 ('beekstraat', 0.9999046921730042),
 ('merelstraat', 0.9999041557312012),
 ('rozenstraat', 0.9999027848243713)]

In [80]:
len(w2v_model.wv.vocab)

15524

## models
* lower_streets_model: window 2, min count 20, grouped on muncipality
* lower_streets_model_2: window 5, min count 2, grouped on muncipality
* streets_model: window 10, min count 2, grouped on postcode
* streets_model_2: window 10, min count 10, grouped on postcode
* geo_model: window 5 min count 2, regionally selected step 5000
* geo_model_2: window 5 min count 2, regionally selected step 500

In [2]:
addresses = pd.read_csv('../data/belgium_addresses.csv')

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,8,9,10,12,13,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
min_x = addresses['EPSG:31370_x'].min()
max_x = addresses['EPSG:31370_x'].max()
min_y = addresses['EPSG:31370_y'].min()
max_y = addresses['EPSG:31370_y'].max()

In [4]:
max_x - min_x

272225.537

In [5]:
max_y - min_y

222241.236

In [6]:
print(addresses.values[0][2])

50.87342697511174


In [7]:
binsize = 1000
coll = {}
nl_id = addresses.columns.get_loc('streetname_nl')
fr_id = addresses.columns.get_loc('streetname_fr')

for row in addresses.values:
    x = (row[0] // binsize) * binsize
    y = (row[1] // binsize) * binsize
    bins = [
        (x, y),
        (x + binsize/2, y),
        (x, y+ binsize/2),
        (x+ binsize/2, y+ binsize/2),
    ]
    for pos in bins:
        if pos not in coll:
            coll[pos] = set()
        try:
            coll[pos].add(row[nl_id].lower())
        except:
            pass
        try:
            coll[pos].add(row[fr_id].lower())
        except:
            pass

In [8]:
blocks = [list(el) for el in coll.values()]

In [9]:
blocks[:10]

[['rue françois lesnino',
  'strijderssquare',
  'rue edmond tollenaere',
  'rue de la sambre',
  'stapelhuisstraat',
  'drootbeekstraat',
  'rue tielemans',
  'rue princesse clémentine',
  'molenbeeksestraat',
  'françois lesninostraat',
  'gustave schildknechtstraat',
  'jean dubrucqlaan',
  'rue de moorslede',
  'claessensstraat',
  'place joseph benoît willems',
  'rue de la comtesse de flandre',
  'koningschapsstraat',
  "rue du champ de l'eglise",
  'square des combattants',
  'rue maurice de moor',
  'boulevard du jubilé',
  'tielemansstraat',
  'ketelsstraat',
  'avenue du port',
  'gravin van vlaanderenstraat',
  'rue marie-christine',
  'zeevaartstraat',
  'albert de meyerstraat',
  'kerkeveldstraat',
  'rue dieudonné lefèvre',
  'richard neyberghlaan',
  'scheldestraat',
  'roodhuisplein',
  'prinses clementinastraat',
  'havenlaan',
  'steenbakkerijstraat',
  'gabrielle petitstraat',
  'avenue prudent bols',
  'dieudonné lefèvrestraat',
  'rue albert de meyer',
  'rue verho

In [10]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

w2v_model = Word2Vec(min_count=2,
                     window=5,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)
w2v_model.build_vocab(blocks, progress_per=10000)

In [11]:
w2v_model.train(blocks, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(25412518, 26897760)

In [12]:
w2v_model.init_sims(replace=True)

In [13]:
w2v_model.save('models/geo_model_2.wv')

In [15]:
w2v_model.most_similar(positive=["dorpsstraat"])

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('romeinse-villadreef', 0.7583727836608887),
 ('anderlechtse-weggevoerdestraat', 0.719359278678894),
 ('engelbert moensstraat', 0.7060449123382568),
 ('gansheidestraat', 0.6964813470840454),
 ('domushof', 0.6935701966285706),
 ('eikeblokplein', 0.6917930245399475),
 ('st. aldegondislaan', 0.690428614616394),
 ("rue d'aumale", 0.6883924007415771),
 ('burgemeester van lindtstraat', 0.686983585357666),
 ('kogelstraat', 0.6863441467285156)]

In [32]:
max(map(len, blocks))

6783